In [1]:
# Changing the working directory
import os
os.chdir('../')

In [19]:
import pandas as pd
import numpy as np
from utils import *
from datetime import date
from skydb.sheets import readSpreadsheet, createSpreadsheet
from skydb.sheets.google_sheets import _get_gc
from grades import getRawGrades

In [3]:
# Getting students from the sky API
students = sky.getUsers()[
        ['student_id', 'id', 'first_name', 'last_name', 'student_info.grade_level']
    ].rename(
        columns={'id':'student_id', 'student_id':'id', 'student_info.grade_level':"grade_level"}
    )

In [ ]:
# Reading the latest School Store data
billing = readSpreadsheet(
        sheet_name = 'School Store'
    ).rename(columns={
        'ID Number':"id"
    })

In [ ]:
# Cleaning the billng data
billing = billing[
        np.isfinite(pd.to_numeric(billing.id, errors='coerce'))
    ].merge(students, 'left')[
        ['student_id', 'first_name', 'last_name', 'grade_level', 'Amount']
    ]

billing.Amount = pd.to_numeric(billing.Amount)

In [ ]:
# Getting the total amount to debt each student
billing = billing.groupby(['student_id', 'first_name', 'last_name', 'grade_level']).sum().reset_index()

In [ ]:
# Reading Smart Tuition family ids
smart = readSpreadsheet(sheet_name = 'ST IDs')

smart.student_id = pd.to_numeric(smart.student_id)

In [ ]:
# Joining data
upload_debts = billing.merge(smart, 'left')

In [ ]:
# Writing the data to a google sheet
timestamp = date.today().strftime("%m/%d/%Y")

sheet_name = "School Store Smart Upload - " + timestamp

spreadsheet = createSpreadsheet(upload_debts.fillna(''), sheet_name = sheet_name)

spreadsheet.share('jmarquez@tampaprep.org', perm_type='user', role='writer')